In [ ]:
import pandas as pd
from datasets import Dataset, Audio

df = pd.read_excel("your_file.xlsx")  # must contain "audio" and "text"
dataset = Dataset.from_pandas(df)
dataset = dataset.cast_column("audio", Audio())


In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small")

def prepare_example(example):
    audio = example["audio"]

    # Get input features
    inputs = processor(audio["array"], sampling_rate=audio["sampling_rate"], return_tensors="pt")
    input_features = inputs.input_features[0]

    # Tokenize transcription
    labels = processor.tokenizer(example["text"], return_tensors="pt").input_ids[0]

    return {
        "input_features": input_features,
        "labels": labels,
    }

# ✅ Use batched=False here (important to avoid the list-of-lists error)
dataset = dataset.map(prepare_example, remove_columns=dataset.column_names)


In [ ]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import torch

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_features": f["input_features"]} for f in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        label_features = [{"input_ids": f["labels"]} for f in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels
        return batch


In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=processor.tokenizer.bos_token_id
)


In [ ]:
from transformers import WhisperForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-fa-finetuned",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    learning_rate=1e-5,
    max_steps=1000,
    save_steps=200,
    logging_steps=25,
    fp16=True,
    report_to="none"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator,
    tokenizer=processor.tokenizer
)


In [ ]:
trainer.train()